In [1]:
import argparse
import logging
import numpy as np
from pathlib import Path
from tqdm import tqdm

import torch
import torchaudio
from torchaudio.functional import resample
from tqdm import tqdm

In [2]:
hubert = torch.hub.load("bshall/hubert:main", f"hubert_soft").cuda()

/home/akorolev/.local/lib/python3.9/site-packages/torch/hub.py:266: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/bshall/hubert/zipball/main" to /home/akorolev/.cache/torch/hub/main.zip
Downloading: "https://github.com/bshall/hubert/releases/download/v0.1/hubert-soft-0d54a1f4.pt" to /home/akorolev/.cache/torch/hub/checkpoints/hubert-soft-0d54a1f4.pt


  0%|          | 0.00/361M [00:00<?, ?B/s]

In [4]:
filelist = [
        "/home/akorolev/master/projects/vits-emotts/filelists/emotional_train.txt",
        "/home/akorolev/master/projects/vits-emotts/filelists/deepph_train_de.txt_cleaned",
        "/home/akorolev/master/projects/vits-emotts/filelists/deepph_train_en.txt_cleaned",
        "/home/akorolev/master/projects/vits-emotts/filelists/deepph_train_pl.txt_cleaned",
        "/home/akorolev/master/projects/vits-emotts/filelists/deepph_train_de_kcd.txt_cleaned",
        "/home/akorolev/master/projects/vits-emotts/filelists/deepph_test.txt_cleaned"
    ]

all_paths = []

for f in filelist:
    with open(f, "r") as rf:
        for line in rf:
            audiofile = line.split("|")[0]
            all_paths.append(audiofile)


for in_path in tqdm(all_paths):
    wav, sr = torchaudio.load(in_path)
    wav = resample(wav, sr, 16000)
    wav = wav.unsqueeze(0).cuda()

    with torch.inference_mode():
        units = hubert.units(wav)

    np.save(in_path.replace(".wav", ".hubert.npy"), units.squeeze().cpu().numpy())

100%|██████████| 264390/264390 [8:45:40<00:00,  8.38it/s]   
